In [1]:
import import_ipynb

In [18]:
import rnn_from_scratch as scratch
import stock_price_predictor as spp

In [6]:
scratch.lstm_modified()

test


In [10]:
data = spp.get_ticker_data('GOOGL', 10)

10y


In [11]:
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-02-10,304.244232,304.369354,302.302307,303.258270,4645750,0,0.0
2012-02-13,305.555542,307.227234,305.315308,306.406403,3628967,0,0.0
2012-02-14,306.076080,306.306305,302.682678,305.185181,3603793,0,0.0
2012-02-15,306.771759,306.771759,301.581573,303.083069,4846948,0,0.0
2012-02-16,301.711700,304.709717,299.164154,303.563568,5056738,0,0.0
...,...,...,...,...,...,...,...
2022-02-03,2914.110107,2993.500000,2853.310059,2861.800049,3575900,0,0.0
2022-02-04,2868.889893,2897.500000,2810.300049,2865.860107,2897200,0,0.0
2022-02-07,2885.000000,2885.000000,2778.000000,2784.020020,2748700,0,0.0


In [12]:
data = spp.clean_data(data)
data = spp.normalize_data(data, ['Open', 'High', 'Low', 'Close', 'Volume'])

X, y = spp.prepare_sets(data)
train_in, train_out, test_in, test_out = spp.split_data(X, y)

X_train, y_train = spp.prepare_sequences(train_in, train_out)
X_test, y_test = spp.prepare_sequences(test_in, test_out)

In [14]:
print(f'{len(X_train)}, {len(X_test)}')
print(f'{X_train.shape}, {y_train.shape}, {X_test.shape}, {y_train.shape}')

1912, 503
(1912, 50, 4), (1912, 1), (503, 50, 4), (1912, 1)


In [35]:
y_train[0]

array([0.00933407])

In [30]:
class LSTM():
    def __init__(self, vocab_size, hidden_size=50):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.z_size = hidden_size + vocab_size
        self.parameters = []
        self.training_loss = []
        self.validation_loss = []
        
    def initialize(self):
        self.parameters = scratch.init_lstm(self.hidden_size, self.vocab_size)
        
    def train(self, train_set, epochs=20):
        self.training_loss, self.parameters = scratch.train_lstm(train_set, self.parameters, self.hidden_size, epochs)
        
    def validate(self, val_set, epochs):
        self.validation_loss = scratch.validate_lstm(val_set, self.parameters, self.hidden_size, epochs)
        
    def test(self, test_set, idx_to_word=None):
        results = scratch.test_lstm(test_set, self.parameters, self.hidden_size, idx_to_word)
        return results

In [24]:
lstm = LSTM(4)

In [25]:
lstm.initialize()

In [32]:
train_set = (X_train, y_train)
lstm.train(train_set, epochs=1)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 4

In [36]:
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

In [37]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def show(self):
        print(self.W)
        print(self.b)

In [38]:
test = attention()

In [40]:
test.build([50, 4])

In [41]:
test.show()

<tf.Variable 'att_weight:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.09407338],
       [ 0.04555698],
       [ 0.0531346 ],
       [-0.04047551]], dtype=float32)>
<tf.Variable 'att_bias:0' shape=(4, 1) dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>
